# Domherren
## Datenimport für Wiagweb 1

Quelle: `W:\Domstifte GS\GS_Domherren_DB.accdb`

- [Literatur](#Literatur)
- [Literaturangaben](#Literaturangaben)
- [Ämter](#Ämter)
- [Klöster](#Klöster)
- Hilfstabellen
  - [`cn_online`](#cn_online)
  - [`cn_officelookup`](#cn_officelookup)
  - [`cn_office.location_show`](#cn_office.location_show)
  - [`cn_era`](#cn_era)
  - [`cn_namelookup`](#cn_namelookup)
  - [`cn_idlookup`](#cn_idlookup)

2021-11-11

In [1]:
# wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"
wds_path="..\\.."

"..\\.."

In [2]:
cd(wds_path)

In [3]:
pwd()

"C:\\Users\\georg\\Documents\\projekte\\WiagDataSetup.jl"

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

In [6]:
using Revise

In [7]:
using WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1317


In [8]:
using MySQL

In [9]:
using DataFrames

In [10]:
Wds = WiagDataSetup

WiagDataSetup

In [108]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg
stdin> topf&deckel


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

In [11]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame; # abgelöst durch Wds.sql_df

Datenbank anlegen: `wiag_in`  
Tabellen anlegen: siehe `domherren_create.sql`
- `cn_canon`
- `cn_office`
- `cn_canon_reference`
- `cn_reference`

## Literatur

In [12]:
sql = "SELECT * FROM domherr.tbl_reference"
df_vol = Wds.sql_df(sql);

In [13]:
names(df_vol)

5-element Vector{String}:
 "ID_Ref"
 "Literatur"
 "Verfasser"
 "OnlineRessource"
 "Kurztitel"

In [14]:
columns = [
    :ID_Ref => :id,
    :Literatur => :title,
    :Verfasser => :author,
    :OnlineRessource => :online_resource,
    :Kurztitel => :shorttitle,
]

5-element Vector{Pair{Symbol, Symbol}}:
          :ID_Ref => :id
       :Literatur => :title
       :Verfasser => :author
 :OnlineRessource => :online_resource
       :Kurztitel => :shorttitle

In [15]:
table_name = "cn_reference"
Wds.filltable!(table_name, select(df_vol, columns), clear_table = true)

┌ Info: Rows inserted: 46
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


46

## Domherren

Der Status kann 'online' oder 'merged' sein, damit Literaturangaben gesammelt werden können.

In [17]:
sql = "SELECT ID_Domherr as id, " *
"Eintrag_Quelle as item_reference, Seite_Quelle as page_reference, Quelle as id_reference, ID_Fremdschlüssel as id_in_reference, " *
"Vorname as givenname, Praefix as prefix_name, Familienname as familyname, " *
"Sterbedatum as date_death, Geburtsdatum as date_birth, " *
"Orden as religious_order, " *
"URL_Wikipedia as wikipedia_url, " *
"fertig as isready, " *
"Bemerkung_Red as annotation_ed, " *
"Bistum as diocese, " *
"AkademischerTitel as academic_title, " *
"GND_ID as gnd_id, " *
"GSN_ID as gsn_id, " *
"VIAF_id as viaf_id," *
"Wikidata_ID as wikidata_id, " *
"Autor_Original as author_orig, " *
"Familienname_Variante as familyname_variant, Vorname_Variante as givenname_variant, " *
"Kommentar_Name as comment_name, Kommentar_Person as comment_person, " *
"BelegdatumBeginn as date_min_in, BelegdatumEnde as date_max_in, " *
"WIAG_ID_Bischof as wiag_episc_id, " *
"Identisch_mit as same_as, Merged_Into as merged_into, " *
"Status as status " *
"FROM domherr.tbl_domherren " *
"WHERE Status = 'online' OR Status = 'merged'"
df_dh = Wds.sql_df(sql);

In [18]:
Wds.clean_up!(df_dh)

In [19]:
size(df_dh)

(3260, 31)

In [24]:
df_r = filter(:id => isequal(10949), df_dh, view = true)
df_r[1, [:id, :status, :givenname, :familyname_variant]]

,id,status,givenname,familyname_variant
,Int32,SubStri…,SubStri…,SubStri…?
88,10949,online,Wilhelm Egon,Fürstenberg-Heiligenberg; Reichsgraf von Fürstenberg-Heiligenberg; Fürst von Fürstenberg-Heiligenberg


Das wird von der Funktion `fillcnnamelookup` abgefangen.

Feld `isready` in Int umwandeln

In [30]:
isready(bit) = ismissing(bit) ? 0 : (bit == MySQL.API.Bit(1) ? 1 : 0)
transform!(df_dh, :isready => ByRow(isready) => :isready);

In [21]:
filter(:isready => isequal(1), df_dh[1:5, [:id, :givenname, :isready]])

,id,givenname,isready
,Int32,SubStri…,Int64
1,10007,Adolf Damian Wolfgang,1
2,10014,Philipp Wilderich,1
3,10084,Franz Wilhelm,1
4,10091,Johann Friedrich Arnold,1
5,10141,Karl Wilhelm Joseph,1


## Literaturangaben

direkte Einträge (alle Einträge haben einen Literaturverweis)

In [31]:
ref1(status, merged_into) = isequal(status, "online") && ismissing(merged_into)
df_dh_ref1 = filter([:status, :merged_into] => ref1, df_dh);

In [32]:
size(df_dh_ref1)

(2430, 31)

In [33]:
columns = [
    :id => :id_canon,
    :id => :id_canon_orig,
    :id_reference => :id_reference,
    :item_reference => :item_reference,
    :page_reference => :page_reference,
    :id_in_reference => :id_in_reference,
];

In [34]:
table_name = "cn_canon_reference"
Wds.filltable!(table_name, select(df_dh_ref1, columns), clear_table = true)

┌ Info: Rows inserted: 2430
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


2430

In [35]:
ref2(status, merged_into) = isequal(status, "merged") && !ismissing(merged_into)
df_dh_ref2 = filter([:status, :merged_into] => ref2, df_dh);

In [36]:
size(df_dh_ref2)

(829, 31)

In [37]:
columns_2nd = [
    :merged_into => :id_canon,
    :id => :id_canon_orig,
    :id_reference => :id_reference,
    :item_reference => :item_reference,
    :page_reference => :page_reference,
    :id_in_reference => :id_in_reference,
];

In [38]:
table_name = "cn_canon_reference"
Wds.filltable!(table_name, select(df_dh_ref2, columns_2nd), clear_table = false)

┌ Info: Rows inserted: 829
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


829

Auszählung zu häufigsten Literaturverweisen
```
+----------+---+
| id_canon | n |
+----------+---+
|    14077 | 5 |
|    18785 | 5 |
|    18727 | 4 |
|    19313 | 4 |
|    18702 | 4 |
|    18677 | 4 |
|    11041 | 4 |
|    19318 | 4 |
|    18770 | 4 |
|    18783 | 4 |
|    11075 | 4 |
|    11007 | 3 |
+----------+---+
```

## Lebensdaten
Hier werden nur noch Einträge benötigt mit Status = 'online'

In [39]:
df_dh_online = filter(:status => isequal("online"), df_dh);

In [40]:
nrow(df_dh), nrow(df_dh_online)

(3260, 2431)

`date_hist_first` ist befüllt, allerdings mit Text.  
Jahreszahlen auslesen.

In [41]:
date_hist_first(date) = Wds.parsemaybe(date, :lower)
transform!(df_dh_online, :date_min_in => (x -> date_hist_first.(x)) => :date_hist_first);

┌ Warning: Could only find year in 
│   s = vor) 1571
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1106


In [42]:
date_hist_last(date) = Wds.parsemaybe(date, :upper)
transform!(df_dh_online, :date_max_in => (x -> date_hist_last.(x)) => :date_hist_last);

In [43]:
numdate_death(date) = Wds.parsemaybe(date, :upper)
transform!(df_dh, :date_death => (x -> numdate_death.(x)) => :numdate_death);

┌ Warning: Could only find year in 
│   s = nach) 1317
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1106
┌ Warning: Could only find year in 
│   s = vor) 1346
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1106


In [44]:
numdate_birth(date) = Wds.parsemaybe(date, :lower)
transform!(df_dh_online, :date_birth => (x -> numdate_birth.(x)) => :numdate_birth);

Die Einträge für Wikipedia-Seiten enthalten zum Teil '#' und sind doppelt.

In [45]:
transform!(df_dh_online, :wikipedia_url => (x -> Wds.fix_Wikipedia_URL.(x)) => :wikipedia_url);

In [46]:
table_name = "cn_canon"
Wds.filltable!(table_name, select(df_dh_online, Not([:date_min_in, :date_max_in])), clear_table = true)

┌ Info: Rows inserted: 2431
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


2431

## Ämter

Betrachte nur Ämter von relevanten Domherren. (`status = 'online'`)

In [47]:
sql = "SELECT ID_Amt as id_in_dh, ID_Domherr as id_canon, ID_Fremdschlüssel as id_in_reference, " *
"Amtsart as office_name, Amtsbeginn as date_start, Amtsende as date_end, " *
"Bistum as diocese, Institution as institution, ID_Kloster as id_monastery, " *
"Profession as profession, Amt_weltlich as office_mundane, Herrschaftsgebiet as dominion, " *
"Ort as location, Archidiakonat as archdeacon_territory, " *
"Dignität as dignity, " *
"Amt_Kommentar as comment, " *
"Sort as sortkey " *
"FROM domherr.tbl_domherren_aemter " *
"WHERE ID_Domherr in (SELECT ID_Domherr FROM domherr.tbl_domherren WHERE Status = 'online')"
df_oc = Wds.sql_df(sql);

In [48]:
size(df_oc)

(5200, 17)

In [49]:
date_start(date) = Wds.parsemaybe(date, :lower)
transform!(df_oc, :date_start => (x -> date_start.(x)) => :numdate_start);

┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1111
┌ Warning: Could only find year in 
│   s = von 1521
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1106
┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1111
┌ Warning: Could not parse 
│   s = wohl im 9. Jahrhundert
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1111
┌ Info: First century date 
│   s = 11. oder 12. Jahrhundert
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1089
┌ Info: First century date 
│   s = 11. oder 12. Jahrhundert
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1089
┌ Info: First century date 
│   s = 11. oder 12. Jahrhundert
└ @ WiagDataSetup C:\Users\georg\Documents\

In [50]:
date_end(date) = Wds.parsemaybe(date, :upper)
transform!(df_oc, :date_end => (x -> date_end.(x)) => :numdate_end);

┌ Warning: Could only find year in 
│   s = vor) 1464
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1106


Welche Diözesen sind in der Datenbank?  
befülle `diocese_in_db`

In [51]:
sql = "SELECT diocese FROM wiag.diocese"
df_dioc = Wds.sql_df(sql);

In [52]:
in_db(diocese) = ismissing(diocese) ? 0 : (in(diocese, df_dioc.diocese) ? 1 : 0)
df_oc[!, :diocese_in_db] .= 0;
for row in eachrow(df_oc)
    row[:diocese_in_db] = in_db(row[:diocese])
end

Einige Klöster sind nicht mehr in der Klosterdatenbank.  
Befülle als Abhilfe die Felder `institution` und `location` und `monastery` mit Dummy-Zeilen (siehe unten)

In [53]:
sql = "SELECT wiagid FROM wiag.monastery"
df_mon_id = Wds.sql_df(sql);

In [57]:
setdiff(skipmissing(df_oc.id_monastery), df_mon_id.wiagid)

1-element Vector{Int32}:
 4066

Domstift Verdun (4066) ist in der Klosterdatenbank (siehe unten)

In [58]:
d_mg_mon = Dict(
    4068 => ("Domstift Kurland, Livland", missing),
    4074 => ("Domstift Reval, Estland", "Reval"),
    4075 => ("Domstift Riga, Lettland", "Riga")
);

In [59]:
df_inst = filter(:institution => !ismissing, df_oc);

In [60]:
size(df_inst)

(0, 20)

In [61]:
for i_mon in keys(d_mg_mon)
    row_idx = isequal.(df_oc.id_monastery, i_mon)
    df_oc[row_idx, :institution] .= d_mg_mon[i_mon][1]
    df_oc[row_idx, :location] .= d_mg_mon[i_mon][2]
end

In [62]:
df_inst = filter(:institution => !ismissing, df_oc);
df_inst[:, [:id_canon, :office_name, :institution, :location]]

,id_canon,office_name,institution,location
,Int32?,String?,String?,String?
1,19489,Domherr,"Domstift Reval, Estland",Reval
2,19405,Domherr,"Domstift Reval, Estland",Reval
3,19054,Domherr,"Domstift Reval, Estland",Reval
4,26602,Domherr,"Domstift Riga, Lettland",Riga
5,26602,Dompropst,"Domstift Kurland, Livland",missing
6,26715,Domherr,"Domstift Riga, Lettland",Riga
7,19649,Domherr,"Domstift Reval, Estland",Reval
8,26709,Domherrenbewerber,"Domstift Reval, Estland",Reval


In [63]:
table_name = "cn_office"
Wds.filltable!(table_name, df_oc, clear_table = true)

┌ Info: Rows inserted: 5200
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


5200

## Klöster
Aus der Klosterdatenbank beziehen, aufgrund der Ämter aller Domherren  
https://api.gs.sub.uni-goettingen.de/v1/monastery/2077

Zu befüllen: `monastery` und `monastery_location`

In [64]:
using HTTP, JSON, DataFrames, MySQL

In [14]:
url_gs_mon = "https://api.gs.sub.uni-goettingen.de/v1/monastery/"

"https://api.gs.sub.uni-goettingen.de/v1/monastery/"

In [103]:
StringMayBe = Vector{Union{String, Missing}}
df_mon = DataFrame(
    wiagid = Int[],
    name = StringMayBe(),
    patrocinium = StringMayBe()
)
df_mon_loc = DataFrame(
    wiagid_monastery = Int[],
    longitude = StringMayBe(),
    latitude = StringMayBe(),
    placeName = StringMayBe(),
    locationName = StringMayBe(),
    beginTPQ = StringMayBe(),
    endTPQ = StringMayBe()
);

IDs relevanter Klöster lesen

In [88]:
sql = "SELECT DISTINCT id_monastery FROM 
(SELECT id_monastery FROM wiag.office WHERE id_monastery IS NOT NULL
UNION SELECT id_monastery FROM wiag_in.cn_office WHERE id_monastery IS NOT NULL
UNION SELECT id_monastery FROM wiag_in.cn_office_gs WHERE id_monastery IS NOT NULL) AS c"
df_id_mon = Wds.sql_df(sql);

In [83]:
size(df_id_mon)

(395, 1)

In [89]:
id_debug = df_id_mon.id_monastery[160:end];
id_all = df_id_mon.id_monastery;

Lies Klöster aus der Klosterdatenbank. Wenn ein Kloster nicht gefunden wird, wird ein Dummy-Eintrag angelegt, um einen Absturz der Web-Anwendung zu vermeiden.

In [104]:
Wds.get_gs_monasteries!(df_mon, df_mon_loc, id_all)

┌ Info: 80
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Info: 160
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Warning: Could not parse  (id: 4068)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1374
┌ Warning: Could not parse  (id: 4074)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1374
┌ Warning: Could not parse  (id: 4075)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1374
┌ Info: 240
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Info: 320
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Warning: Could not parse  (id: 7203)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1374


Info

In [72]:
size(df_mon)

(395, 3)

In [99]:
df_mon[1:7, :]

,wiagid,name
,Int64,String?
1,415,"Benediktinerabtei Corvey, kurzzeitig Domstift"
2,415,"Benediktinerabtei Corvey, kurzzeitig Domstift"
3,791,"Benediktinerkloster St. Michael, Lüneburg"
4,3294,"Benediktinerkloster, dann Kollegiatstift St. Leodegar, Murbach, Frankreich"
5,3428,"Stift Haug, Würzburg"
6,3560,"Kollegiatstift St. Pauli, Bremen, später Benediktinerkloster"
7,4623,"Benediktinerreichsabtei Stablo-Malmedy, Belgien"


In [105]:
df_mon[!, :id_monastery] .= 1:nrow(df_mon);

In [99]:
names(df_mon)

4-element Vector{String}:
 "wiagid"
 "name"
 "patrocinium"
 "id_monastery"

In [106]:
rename!(df_mon, :name => :monastery_name);

In [109]:
table_name = "monastery"
Wds.filltable!(table_name, df_mon, clear_table = true)

┌ Info: Rows inserted: 395
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


395

In [110]:
df_mon_loc[100:105, :]

,wiagid_monastery,longitude,latitude,placeName,locationName
,Int64,String?,String?,String?,String?
1,982,8.108056,51.571528,Soest,missing
2,983,8.10516666666667,51.5708333333333,Soest,missing
3,1015,10.573333333333,49.302777777778,Ansbach,missing
4,1016,9.146389,49.973611,Aschaffenburg,missing
5,1043,9.1744,47.6625,Konstanz,missing
6,1051,7.3012,50.18369,Treis-Karden,Karden an der Mosel


In [117]:
df_mg_geo = filter(:longitude => ismissing, df_mon_loc);
size(df_mg_geo)

(0, 7)

In [111]:
parseFloat(x) = parse(Float64, x)
transform!(df_mon_loc, :longitude => ByRow(parseFloat) => :longitude);

In [112]:
transform!(df_mon_loc, :latitude => ByRow(parseFloat) => :latitude);

In [122]:
names(df_mon_loc)

7-element Vector{String}:
 "wiagid_monastery"
 "longitude"
 "latitude"
 "placeName"
 "locationName"
 "beginTPQ"
 "endTPQ"

In [124]:
df_mg_place_name = filter(:placeName => ismissing, df_mon_loc);
size(df_mg_place_name)

(0, 7)

In [113]:
df_mon_loc[!, :id] .= 1:nrow(df_mon_loc);

In [114]:
columns = [
    :id => :id,
    :wiagid_monastery => :wiagid_monastery,
    :longitude => :longitude,
    :latitude => :latitude,
    :placeName => :place_name,
    :locationName => :location_name,
    :beginTPQ => :location_begin_tpq,
    :endTPQ => :location_end_tpq
];

In [115]:
table_name = "monastery_location"
Wds.filltable!(table_name, select(df_mon_loc, columns), clear_table = true)

┌ Info: Rows inserted: 422
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


422

## Hilfstabellen
### `cn_online`
``` sql
CREATE TABLE cn_online LIKE wiag.cn_online;
```

Domherren aus der Personendatenbank

In [118]:
sql = "SELECT id as id_gs, gsn_id, givenname, familyname FROM wiag_in.cn_canon_gs WHERE status = 'online'"
df_pub_gs = Wds.sql_df(sql);

In [119]:
size(df_pub_gs)

(7510, 4)

In [14]:
size(unique(df_pub_gs.gsn_id))

(7510,)

Domherren aus der Domherrendatenbank

In [121]:
sql = "SELECT id as id_dh, wiag_episc_id as wiagid, gsn_id, givenname, familyname " *
"FROM wiag_in.cn_canon WHERE status = 'online'"
df_pub_dh = Wds.sql_df(sql);

In [122]:
size(df_pub_dh)

(2431, 5)

Prüfe auf GSN-Dubletten

In [123]:
df_dh_gsn = dropmissing(df_pub_dh, :gsn_id);

In [124]:
size(df_dh_gsn)

(399, 5)

In [125]:
dfg = groupby(df_dh_gsn, :gsn_id);
dfg_n = combine(dfg, nrow);

In [126]:
dfg_nth2 = filter(:nrow => (x -> x > 1), dfg_n)

,gsn_id,nrow
,String,Int64


In [127]:
df_dh_gsn_db = innerjoin(df_dh_gsn, dfg_nth2, on = :gsn_id);
sort(select(df_dh_gsn_db, Not(:wiagid)), :gsn_id)

,id_dh,gsn_id,givenname,familyname,nrow
,Int32,String,String?,String?,Int64


Keine Dopplungen merh (2021-11-24), sonst: Die Dopplung bleibt beim weiteren Import erhalten.

Wir gehen davon aus, dass für alle Domherren der Eintrag in der Bischofsdatenbank recherchiert wurde. Wenn es einen solchen Eintrag gibt, wird die WIAG-ID übernommen und `id_ep` wird entsprechend gesetzt. Für die restlichen wird eine WIAG_ID auf der Basis der ID in der Domherrendatenbank vergeben.

In [128]:
rgx_episc = r"EPISCGatz-([0-9]+)-"
function id_ep(wiagid)
    if ismissing(wiagid)
        return missing
    end
    rgm = match(rgx_episc, wiagid)
    if !isnothing(rgm) 
        id_ep = parse(Int, rgm[1])
        return id_ep
    else
        @warn "invalid wiagid " wiagid
        return missing
    end
end

id_ep (generic function with 1 method)

In [129]:
transform!(df_pub_dh, :wiagid => (x -> id_ep.(x)) => :id_ep);

In [130]:
wiag(id) = "WIAG-Pers-CANON-" * lpad(id, 5, "0") * "-001"
for row in eachrow(df_pub_dh)
    if ismissing(row[:wiagid])
        row[:wiagid] = wiag(row[:id_dh])
    end
end

In [131]:
df_pub_dh[120:121, [:id_dh, :id_ep, :wiagid, :givenname, :familyname]]

,id_dh,id_ep,wiagid,givenname,familyname
,Int32,Int64?,String?,String?,String?
1,10997,10465,WIAG-Pers-EPISCGatz-10465-001,Otto Matthias,Mallinckrodt
2,10998,missing,WIAG-Pers-CANON-10998-001,Raban Wilhelm,Malsburg


Schnittmenge von Personendatenbank und Domherrendatenbank

In [132]:
df_pub_dh_gsn = filter(:gsn_id => !ismissing, df_pub_dh)
df_pub_both = innerjoin(df_pub_dh_gsn, select(df_pub_gs, :id_gs, :gsn_id), on = :gsn_id);

In [133]:
size(df_pub_both) # waren zuletzt 171

(333, 7)

In [28]:
names(df_pub_both)

7-element Vector{String}:
 "id_dh"
 "wiagid"
 "gsn_id"
 "givenname"
 "familyname"
 "id_ep"
 "id_gs"

In [134]:
df_pub_dh_dh = antijoin(df_pub_dh, df_pub_both, on = :id_dh);

In [135]:
size(df_pub_dh_dh) # 2431 - 333 = 2098

(2098, 6)

In [136]:
df_pub_gs_gs = antijoin(df_pub_gs, df_pub_both, on = :id_gs);

In [137]:
size(df_pub_gs_gs) # 7510 - 333 = 7177; 

(7177, 4)

Einträge bei den Bischöfen für Domherren, die nur in der Personendatenbank vorkommen

In [139]:
sql = "SELECT wiagid as id_ep, gsid as gsn_id, givenname as givenname_ep, familyname as familyname_ep " *
"FROM wiag.person " *
"WHERE gsid IS NOT NULL AND gsid <> ''"
df_ep = Wds.sql_df(sql);

In [140]:
df_pub_gs_gs_ep = leftjoin(df_pub_gs_gs, select(df_ep, :id_ep, :gsn_id), on = :gsn_id);

In [141]:
size(df_pub_gs_gs_ep)

(7177, 5)

WIAG-ID erstellen

In [142]:
function wiag_id4gs(id_ep, id_gs)
    if !ismissing(id_ep)
        return "WIAG-Pers-EPISCGatz-" * lpad(id_ep, 5, "0") * "-001"
    else
        return "WIAG-Pers-CANON-" * lpad(id_gs, 5, "0") * "-001"
    end
end

wiag_id4gs (generic function with 1 method)

In [143]:
transform!(df_pub_gs_gs_ep, [:id_ep, :id_gs] => ((x, y) -> wiag_id4gs.(x, y)) => :wiagid);

In [44]:
names(df_pub_both)

7-element Vector{String}:
 "id_dh"
 "wiagid"
 "gsn_id"
 "givenname"
 "familyname"
 "id_ep"
 "id_gs"

`cn_online` befüllen

In [144]:
table_name = "cn_online"
Wds.filltable!(table_name, select(df_pub_both, Not(:gsn_id)), clear_table = true)

┌ Info: Rows inserted: 333
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


333

In [145]:
Wds.filltable!(table_name, select(df_pub_dh_dh, Not(:gsn_id)), clear_table = false)

┌ Info: Rows inserted: 2098
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


2098

In [146]:
Wds.filltable!(table_name, select(df_pub_gs_gs_ep, Not(:gsn_id)), clear_table = false)

┌ Info: Rows inserted: 7177
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


7177

SQL
``` sql
select count(*) from cn_online;
```
9608

### `cn_namelookup`
``` sql
CREATE TABLE cn_namelookup LIKE wiag.cn_namelookup;
```

In [147]:
Wds.fillcnnamelookup("cn_namelookup", "cn_online", "cn_canon", "cn_canon_gs")

┌ Info: clear 
│   tbllookup = cn_namelookup
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:689
┌ Info: 400
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info: 800
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info: 1200
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info: 1600
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info: 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info: 2400
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info: 2800
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info: 3200
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:751
┌ Info:

9608

### `cn_office.location_show`

In [148]:
Wds.fillofficelocation("cn_office", "monastery_location", "wiag.place", "cn_online", "id_dh")

┌ Info: 1000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:563
┌ Info: 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:563
┌ Info: 3000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:563
┌ Info: 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:563
┌ Warning: ("No place found for office", 42220)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:554
┌ Info: 5000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:563


5189

4 Orte können nicht bestimmt werden, weil die Amtszeit nicht zu einem angegebenen Zeitraum für das Kloster passt. (2021-11-23)

### `cn_officelookup`

Daten aus der Domherrendatenbank

`cn_office.id_monastery` bezieht sich auf `monastery.wiagid` nicht mehr auf `monastery.id_monastery`. `monastery.wiagid` ist gleichzeitig die Nummer in der Klosterdatenbank.

In [149]:
sql = "SELECT co.id AS id_online, office_name, " *
"location_show AS location_name, id_monastery, archdeacon_territory, " *
"o.numdate_start AS numdate_start, o.numdate_end AS numdate_end, " *
"ds.name as domstift " *
"FROM cn_office AS o " * 
"JOIN cn_online AS co ON o.id_canon = co.id_dh " *
"LEFT JOIN wiag.domstift AS ds ON o.id_monastery = ds.gs_id"
df_ol_dh = Wds.sql_df(sql);

In [150]:
size(df_ol_dh)

(5200, 8)

In [151]:
table_name = "cn_officelookup"
Wds.filltable!(table_name, df_ol_dh, clear_table = true)

┌ Info: Rows inserted: 5200
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


5200

In [152]:
sql = "SELECT co.id AS id_online, office_name, " *
"location_show AS location_name, id_monastery, archdeacon_territory, " *
"o.numdate_start AS numdate_start, o.numdate_end AS numdate_end, " *
"ds.name as domstift " *
"FROM cn_office_gs AS o " * 
"JOIN cn_online AS co ON o.id_canon = co.id_gs " *
"LEFT JOIN wiag.domstift AS ds ON o.id_monastery = ds.gs_id"
df_ol_gs = Wds.sql_df(sql);

In [153]:
df_ol_gs_ds = filter(:domstift => !ismissing, df_ol_gs);

In [154]:
df_ol_gs_ds[20:26, [:id_online, :office_name, :domstift]]

,id_online,office_name,domstift
,Int32,String?,String?
1,24464,Domherr,Würzburg
2,24466,Domdekan,Bamberg
3,24467,Domherr,Bamberg
4,24468,Domherr,Bamberg
5,24469,Domherr,Bamberg
6,24470,Dompropst,Bamberg
7,24471,Domherr,Bamberg


In [155]:
table_name = "cn_officelookup"
Wds.filltable!(table_name, df_ol_gs, clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1179
┌ Info: Rows inserted: 13002
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


13002

### `cn_era`
Struktur: `id_online, domstift, era_start, era_end`  
Es gibt globale Einträge mit `domstift = 'all'` und Einträge pro Domstift. Diese fassen jeweils den Zeitraum pro Domstift zusammen und schließen Einträge ohne Datumsangabe aus. In Doctrine lässt sich das nicht formulieren, da keine Tabelle verknüpft werden kann (join), die aus einer eigenen Abfrage mit Gruppierung entsteht.

global

Personendatenbank

In [156]:
sql = "SELECT co.id AS id_online, co.id_gs, date_hist_first AS era_start, date_hist_last AS era_end, " *
"numdate_birth AS numdate_birth_gs, numdate_death AS numdate_death_gs " *
"FROM cn_online AS co JOIN cn_canon_gs AS c ON co.id_gs = c.id " *
"WHERE status = 'online'"
df_era_gs = Wds.sql_df(sql);

Amtsdaten

In [157]:
sql = "SELECT id_canon as id_gs, min(numdate_start) AS numdate_start_gs, max(numdate_end) AS numdate_end_gs " *
"FROM cn_office_gs GROUP BY id_canon"
df_o_gs = Wds.sql_df(sql);

In [158]:
size(df_era_gs), size(df_o_gs)

((7510, 6), (7441, 3))

In [159]:
df_era_gs = leftjoin(df_era_gs, df_o_gs, on = :id_gs);

In [160]:
size(df_era_gs)

(7510, 8)

Domherrendatenbank

In [161]:
sql = "SELECT co.id AS id_online, co.id_dh, date_hist_first, date_hist_last, " *
"numdate_birth AS numdate_birth_dh, numdate_death AS numdate_death_dh " *
"FROM cn_online AS co JOIN cn_canon AS c ON co.id_dh = c.id " *
"WHERE status = 'online'"
df_era_dh = Wds.sql_df(sql);

Amtsdaten

In [162]:
sql = "SELECT id_canon as id_dh, min(numdate_start) AS numdate_start_dh, max(numdate_end) AS numdate_end_dh " *
"FROM cn_office GROUP BY id_canon"
df_o_dh = Wds.sql_df(sql);

In [163]:
size(df_era_dh), size(df_o_dh)

((2431, 6), (2431, 3))

In [164]:
df_era_dh = leftjoin(df_era_dh, df_o_dh, on = :id_dh);

Zusammenführen

In [165]:
df_era = outerjoin(df_era_gs, df_era_dh, on = :id_online);

In [166]:
size(df_era)

(9608, 15)

Finde Minima und Maxima

In [167]:
function valid_max(x, x_cmp...)
    for x_i in x_cmp
        x = ismissing(x_i) ? x : (ismissing(x) ? x_i : max(x, x_i))
    end
    return x
end
function valid_min(x, x_cmp...)
    for x_i in x_cmp
        x = ismissing(x_i) ? x : (ismissing(x) ? x_i : min(x, x_i))
    end
    return x
end

valid_min (generic function with 1 method)

In [168]:
col_min = [:era_start, :numdate_start_gs, :numdate_start_dh, :date_hist_first]
transform!(df_era, col_min => ByRow(valid_min) => :era_start);

In [169]:
col_max = [:era_end, :numdate_end_gs, :numdate_end_dh, :date_hist_last]
transform!(df_era, col_max => ByRow(valid_max) => :era_end);

Info

In [172]:
df_info = filter(:id_online => in(22614:22619), df_era);

In [173]:
df_info[:, [:id_online, :id_dh, :id_gs, :numdate_birth_gs, :numdate_birth_dh, col_min...]]

,id_online,id_dh,id_gs,numdate_birth_gs,numdate_birth_dh,era_start,numdate_start_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,22619,missing,81501,missing,missing,1450,1450
2,22618,missing,81502,missing,missing,1155,1155
3,22617,missing,81503,missing,missing,1376,1376
4,22616,missing,81504,missing,missing,1396,1396
5,22615,missing,81505,missing,missing,1183,1196
6,22614,missing,81508,missing,missing,1390,1390


In [174]:
ne_maybe(x, y) = (ismissing(x) || ismissing(y)) ? false : x != y
df_info = filter([:era_start, :numdate_start_dh] => ne_maybe, df_era);

In [175]:
df_info[12:18, [:id_online, :id_dh, :id_gs, col_min...]]

,id_online,id_dh,id_gs,era_start,numdate_start_gs,numdate_start_dh,date_hist_first
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,19007,10919,81359,1738,1756,1756,missing
2,19008,20369,81379,1371,1371,1408,missing
3,19009,10885,81391,1734,1758,1752,1752
4,19011,10887,81397,1714,1759,1732,missing
5,19012,10896,81428,1683,1683,1716,missing
6,19016,10915,81447,1702,1702,1746,missing
7,19019,10934,81572,1732,1755,1755,1755


In [176]:
ne_maybe(x, y) = (ismissing(x) || ismissing(y)) ? false : x != y
df_info = filter([:era_end, :numdate_end_gs] => ne_maybe, df_era);

In [177]:
df_info[12:18, [:id_online, :id_dh, :id_gs, col_max...]]

,id_online,id_dh,id_gs,era_end,numdate_end_gs,numdate_end_dh,date_hist_last
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,19017,15306,81453,1699,1665,1699,1699
2,19018,20382,81506,1432,1429,1432,missing
3,19019,10934,81572,1783,1761,1761,1761
4,19025,10944,81633,1825,1802,1789,missing
5,19029,10965,81795,1804,1797,1804,1804
6,19036,11019,81956,1777,1772,1777,missing
7,19038,11025,82013,1775,1734,1775,missing


Geburts- und Sterbedatum  
Geburds- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [178]:
function era_birth(era, b1, b2)
    b = valid_min(b1, b2)
    return ismissing(b) ? era : b
end
function era_death(era, b1, b2)
    b = valid_max(b1, b2)
    return ismissing(b) ? era : b
end
    

era_death (generic function with 1 method)

In [179]:
transform!(df_era, [:era_start, :numdate_birth_dh, :numdate_birth_gs] => ByRow(era_birth) => :era_start);

In [180]:
transform!(df_era, [:era_end, :numdate_death_dh, :numdate_death_gs] => ByRow(era_death) => :era_end);

Info

In [181]:
df_info = filter([:era_start, :numdate_birth_dh] => ne_maybe, df_era);

In [182]:
df_info[!, [:id_online, :id_dh, :id_gs, :numdate_birth_gs, :numdate_birth_dh, col_min...]]

,id_online,id_dh,id_gs,numdate_birth_gs,numdate_birth_dh,era_start,numdate_start_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,19007,10919,81359,1738,1742,1738,1756
2,19181,11021,85597,1735,1740,1735,1756
3,19188,10984,85858,1746,1752,1746,1770
4,19225,11054,86880,1750,1756,1750,1760


In [183]:
df_info = filter(:numdate_birth_gs => ismissing, df_era);

In [184]:
df_info[100:105, [:id_online, :id_dh, :id_gs, :numdate_birth_gs, :numdate_birth_dh, col_min...]]

,id_online,id_dh,id_gs,numdate_birth_gs,numdate_birth_dh,era_start,numdate_start_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,19104,26590,84330,missing,missing,1337,1337
2,19105,13714,84332,missing,missing,1237,1240
3,19106,20412,84333,missing,missing,1301,1301
4,19107,20482,84334,missing,missing,1294,1311
5,19108,26580,84335,missing,missing,1247,1247
6,19109,20460,84337,missing,missing,1252,1260


In [185]:
df_info = filter([:era_end, :numdate_death_dh] => ne_maybe, df_era);

In [186]:
df_info[:, [:id_online, :id_dh, :id_gs, :numdate_death_gs, :numdate_death_dh, col_max...]]

,id_online,id_dh,id_gs,numdate_death_gs,numdate_death_dh,era_end,numdate_end_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?


In [187]:
df_info = filter(:numdate_death_gs => ismissing, df_era);

In [188]:
df_info[100:105, [:id_online, :id_dh, :id_gs, :numdate_death_gs, :numdate_death_dh, col_max...]]

,id_online,id_dh,id_gs,numdate_death_gs,numdate_death_dh,era_end,numdate_end_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,19128,20434,84372,missing,missing,1341,missing
2,19130,26583,84375,missing,missing,1342,1342
3,19131,13138,84376,missing,missing,1316,missing
4,19132,19318,84380,missing,missing,1383,1383
5,19133,13752,84382,missing,missing,1288,missing
6,19135,20168,84384,missing,missing,1381,1381


Ersetze fehlende Werte durch den jeweils anderen Wert, falls vorhanden. (NULL erscheint in Sortierungen immer als erstes)

In [189]:
function replace_missing(a, b)
    if ismissing(a)
        a = b
    elseif ismissing(b)
        b = a
    end
    return a, b
end

replace_missing (generic function with 1 method)

In [190]:
transform!(df_era, [:era_start, :era_end] => ByRow(replace_missing) => [:era_start, :era_end]);

Schreibe Daten für `domstift = 'all'`

In [191]:
df_era[!, :domstift] .= "all";

In [192]:
table_name = "cn_era"
Wds.filltable!(table_name, select(df_era, :id_online, :domstift, :era_start, :era_end), clear_table = true)

┌ Info: Rows inserted: 9608
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


9608

Zeitraum pro Domstift

Personendatenbank

In [193]:
sql_union = "SELECT ol_1.id AS id_online, id_canon, d_1.name as domstift, numdate_start, numdate_end " *
"FROM cn_office_gs AS o_gs " *
"JOIN wiag.domstift AS d_1 on o_gs.id_monastery = d_1.gs_id JOIN cn_online AS ol_1 ON ol_1.id_gs = o_gs.id_canon " *
"UNION (SELECT ol.id AS id_online, id_canon, d.name as domstift, numdate_start, numdate_end " *
"FROM cn_office AS o " *
"JOIN wiag.domstift as d on o.id_monastery = d.gs_id JOIN cn_online AS ol ON ol.id_dh = o.id_canon)"
df_od = Wds.sql_df(sql_union);

In [194]:
size(df_od)

(12853, 5)

Info

In [196]:
sql_dh = "SELECT ol.id AS id_online, id_canon, d.name as domstift, numdate_start, numdate_end " *
"FROM cn_office AS o " *
"JOIN wiag.domstift as d on o.id_monastery = d.gs_id JOIN cn_online AS ol ON ol.id_dh = o.id_canon "
df_od_dh = Wds.sql_df(sql_dh);

In [197]:
size(df_od_dh)

(4139, 5)

In [199]:
filter(:id_online => isequal(22501), df_od)

,id_online,id_canon,domstift,numdate_start,numdate_end
,Int32,Int32?,String?,Int32?,Int32?
1,22501,81671,Osnabrück,1426,1457


In [200]:
sql = "SELECT id_online, domstift, min(numdate_start) AS era_start, max(numdate_end) AS era_end " *
"FROM (" * sql_union * ") AS U " *
"GROUP BY id_online, domstift"
df_era_d = Wds.sql_df(sql);

In [201]:
size(df_era_d), length(unique(df_era_d.id_online)), length(unique(df_era_d.domstift))

((11519, 4), 9488, 34)

Info

In [202]:
filter(:id_online => isequal(22501), df_era_d)

,id_online,domstift,era_start,era_end
,Int32,String?,Int32?,Int32?
1,22501,Osnabrück,1426,1457


Ersetze fehlende Werte durch den jeweils anderen Wert, falls vorhanden. (NULL erscheint in Sortierungen immer als erstes)

In [203]:
transform!(df_era_d, [:era_start, :era_end] => ByRow(replace_missing) => [:era_start, :era_end]);

Schreibe Daten für die einzelnen Domstifte

In [204]:
table_name = "cn_era"
Wds.filltable!(table_name, select(df_era_d, :id_online, :domstift, :era_start, :era_end), clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1179
┌ Info: Rows inserted: 11519
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


11519

### `cn_idlookup`

Liese relevante Domherren aus `cn_online`

In [206]:
sql = "SELECT id as id_online, id_dh FROM cn_online WHERE id_dh IS NOT NULL"
df_online_dh = Wds.sql_df(sql);

In [208]:
sql = "SELECT * FROM cn_canon"
df_dh = Wds.sql_df(sql);

Externe Identifier für Domherren aus der Domherrendatenbank

In [209]:
df_idol = DataFrame(:id_dh => Int[], :authority_id => String[])

,id_dh,authority_id
,Int64,String


In [210]:
for row in eachrow(df_dh)
    for col in (:gnd_id, :gsn_id, :viaf_id, :wikidata_id, :wiag_episc_id)
        if !ismissing(row[col])
            push!(df_idol, (row[:id], row[col]))
        end
    end
end

In [211]:
size(df_idol)

(1670, 2)

In [212]:
df_idol = leftjoin(df_idol, df_online_dh, on = :id_dh);

In [213]:
table_name = "cn_idlookup"
Wds.filltable!(table_name, select(df_idol, :id_online, :authority_id), clear_table = true)

┌ Info: Rows inserted: 1670
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


1670

In [216]:
sql = "SELECT id as id_online, id_gs FROM cn_online WHERE id_gs IS NOT NULL"
df_online_gs = Wds.sql_df(sql);

In [218]:
sql = "SELECT * FROM cn_canon_gs WHERE status = 'online'"
df_gs = Wds.sql_df(sql);

In [219]:
size(df_gs)

(7510, 34)

In [220]:
df_idol = DataFrame(:id_gs => Int[], :authority_id => String[])

,id_gs,authority_id
,Int64,String


In [221]:
for row in eachrow(df_gs)
    for col in (:gnd_id, :gsn_id, :viaf_id, :wikidata_id, :wiag_episc_id)
        if !ismissing(row[col])
            push!(df_idol, (row[:id], row[col]))
        end
    end
end

In [222]:
size(df_idol)

(8376, 2)

In [223]:
df_idol = leftjoin(df_idol, df_online_gs, on = :id_gs);

In [224]:
size(df_idol)

(8376, 3)

In [225]:
table_name = "cn_idlookup"
Wds.filltable!(table_name, select(df_idol, :id_online, :authority_id), clear_table = false)

┌ Info: Rows inserted: 8376
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


8376

Daten in die Zieldatenbank übernehmen (im eingespielten Prozess können die Tabellen auch direkt vom Notebook aus beschrieben werden)
```sql
use wiag;
delete from cn_reference;
insert into cn_reference (select * from wiag_in.cn_reference);

delete from cn_canon;
insert into cn_canon (select * from wiag_in.cn_canon);
delete from cn_canon_reference;
insert into cn_canon_reference (select * from wiag_in.cn_canon_reference);
delete from cn_office;
insert into cn_office (select * from wiag_in.cn_office);

delete from cn_online;
insert into cn_online (select * from wiag_in.cn_online);
delete from cn_namelookup;
insert into cn_namelookup (select * from wiag_in.cn_namelookup);

drop table cn_officelookup;
create table cn_officelookup like wiag_in.cn_officelookup;
insert into cn_officelookup (select * from wiag_in.cn_officelookup);

drop table cn_era;
create table cn_era like wiag_in.cn_era;
insert into cn_era (select * from wiag_in.cn_era);

drop table cn_idlookup;
create table cn_idlookup like wiag_in.cn_idlookup;
insert into cn_idlookup (select * from wiag_in.cn_idlookup);

drop table monastery;
create table monastery like wiag_in.monastery;
insert into monastery (select * from wiag_in.monastery);

drop table monastery_location;
create table monastery_location like wiag_in.monastery_location;
insert into monastery_location (select * from wiag_in.monastery_location);
```